# Process Files

Iremos realizar um pré-processamento nos dados.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from featureExtraction import featuresFromDataframe
from readAccelerometerData import createDatasetFromFiles

In [2]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Criando dataframe com os dados
dataf = createDatasetFromFiles()

# extracts from accelerometer data (0) and maximum frequency 32, without filtering
feats0_32 = featuresFromDataframe(dataf, 0, maxfr=32, filtering=False)

#Criando dataframe
df_features = pd.DataFrame(feats0_32)

#Sort names
df_features.sort_values(0,inplace=True)

#Criando encoder
le = LabelEncoder().fit(df_features[0])

#Codificando o nome das pessoas
df_features[0] = ['person_' + str(ID) for ID in le.transform(df_features[0])]

#Salvando dados
df_features.to_csv('features.csv',index=False)

O dataframe df_features possui colunas no seguinte formato:

- Nome da pessoa
- Número da avaliação em que se tomou a droga 1
- Droga 1 (64 colunas):
    -  Medidas (8 colunas):
        - Features (18 colunas).
- Droga 2 (64 colunas):
    - Medidas (8 colunas):
        - Features (18 colunas).
        
Cada linha corresponde a uma pessoa.

In [3]:
#Dataframe com os dados transformados
df_final = pd.DataFrame()

#Range das colunas com atributos da serie temporal
start = 2
end = 20
step = 18

#Roda para os 16 arquivos
for i in range(16):
    df_final = df_final.append(pd.DataFrame(df_features.iloc[:,start:end].values))

    #Atualiza range para pegar outro arquivo
    start+=step
    end+=step

#Setando nome para as colunas
df_final.columns = ['pse', 'pspf1', 'pspf2', 'pspf3', 'psp1', 'psp2', 'psp3', 'wpsf','RMS','range','mean','var','skew','kurt','jerk_mean','jerk_std','jerk_skew','jerk_kur']

In [4]:
#Dataframe com os dados transformados
df_final = pd.DataFrame()

#Range das colunas com atributos da serie temporal
start = 2
end = 20
step = 18

#Roda para os 16 arquivos
for i in range(16):
    df_final = df_final.append(pd.DataFrame(df_features.iloc[:,start:end].values))

    #Atualiza range para pegar outro arquivo
    start+=step
    end+=step

#Setando nome para as colunas
df_final.columns = ['pse', 'pspf1', 'pspf2', 'pspf3', 'psp1', 'psp2', 'psp3', 'wpsf','RMS','range','mean','var','skew','kurt','jerk_mean','jerk_std','jerk_skew','jerk_kur']

#Criando colunas nome, avaliacao, tipo de droga tomada e measure
df_final['name'] = np.tile(df_features.iloc[:,0],16)
df_final['evaluate'] = np.tile(df_features.iloc[:,1],16)
df_final['measure'] = np.tile(np.repeat([1,2,3,4,5,6,7,8],22),2)
df_final['drug'] = np.repeat([1,2],176)

#Sort
df_final.sort_values(by=['name','drug','measure'],inplace=True)

#Mudando tipo das colunas
df_final.loc[:,['pspf1','pspf2','pspf3']] = df_final[['pspf1','pspf2','pspf3']].astype('int')

#Mapeando droga
df_final.loc[:,'drug'] = df_final['drug'].map({1:'placebo',2:'CBD'})

#Filtro para a avaliacao
filter_evaluate = ((df_final['evaluate'] == 1) & (df_final['drug'] == 'placebo')) | ((df_final['evaluate'] == 2) & (df_final['drug'] == 'CBD'))

#Mapeando medida
df_final['measure'] = df_final['measure'].map({1:'B',3:'P',4:'A',5:'S1',6:'S2',7:'F1',8:'F2'})

#Setando dados - Agora evaluate indica o numero da avalicao
df_final.loc[filter_evaluate,'evaluate'] = 1
df_final.loc[~filter_evaluate,'evaluate'] = 2

#Filtrando - exclusao da medida 2 por nao ser relevante segundo os medicos
df_final.dropna(inplace=True)

#Nome como index
df_final.set_index('name',inplace=True)

#Salvando dados
df_final.loc[:,['pse','pspf1','pspf2','pspf3','psp1','psp2','psp3','wpsf','evaluate','measure','drug']].to_csv('parkinson_v1.csv')
df_final.to_csv('parkinson_v2.csv')

# Observações

- Os scripts <b>featureExtraction.py</b> e <b>readAccelerometerData.py</b> foram desenvolvidos pelo professor Moacir e seus códigos originais se encontram em https://github.com/maponti/AccelerometerAnalysisParkinsonsCannabidiol. 

- Os dados 'brutos' não se encontram neste repositório por questões de confidencialidade das pessoas envolvidas no experimento. Assim não é possível executar este notebook.